In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, BooleanType, FloatType

In [ ]:
spark = SparkSession.builder.appName("spark_sql_app").getOrCreate()

# Read DataFrame

In [ ]:
schema = StructType(
        [
            StructField("date", StringType(), True),
            StructField("delay", IntegerType(), True),
            StructField("distance", IntegerType(), True),
            StructField("origin", StringType(), True),
            StructField("destination", StringType(), True),
        ]
    )

df = spark.read.format("csv").schema(schema).option("header", "true").load("/opt/bitnami/spark/custom_data/chapter4/departuredelays.csv")
df.show()

# Create temporary view

In [ ]:
spark.catalog.dropTempView("us_delay_flights_tbl")
df.createOrReplaceTempView("us_delay_flights_tbl")

# SQL Operations

In [ ]:
spark.sql("""
SELECT distance, origin, destination
FROM us_delay_flights_tbl WHERE distance > 1000
ORDER BY distance DESC
""").show(10)

In [ ]:
spark.sql("""
SELECT date, distance, origin, destination
FROM us_delay_flights_tbl WHERE delay > 120 AND
origin = 'SFO' and destination='ORD' 
ORDER BY delay DESC
""").show(10)

In [ ]:
spark.sql("""
SELECT delay, origin, destination,
CASE
    WHEN delay>360 THEN 'Very Long Delays'
    WHEN delay>=120 and delay<=360 THEN 'Long Delays'
    WHEN delay>=60 and delay <120 THEN 'Short Delays'
    WHEN delay>0 and delay<60 THEN  'Tolerable Delays'
    WHEN delay=0 THEN 'No Delays'
END as Flight_Delays
FROM us_delay_flights_tbl
ORDER BY origin, delay DESC
""").show(10)

# SQL databases and tables 

In [ ]:
spark.sql("""CREATE DATABASE IF NOT EXISTS learn_spark_db""")
spark.sql("""use learn_spark_db""")

## Managed table

In [ ]:
spark.sql("""DROP TABLE IF EXISTS managed_us_delay_flights_tbl""")
df.write.mode('overwrite').saveAsTable("managed_us_delay_flights_tbl")

## Unmanaged table

In [ ]:
spark.sql("""DROP TABLE IF EXISTS unmanaged_us_delay_flights_tbl""")
df.write.option("path", "/opt/bitnami/spark/custom_data/chapter4/output/").mode('overwrite').saveAsTable("unmanaged_us_delay_flights_tbl")

## View metadata

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.catalog.listColumns("unmanaged_us_delay_flights_tbl")

In [ ]:
df_table = spark.table("us_delay_flights_tbl")
df_table.printSchema()

In [ ]:
spark.stop()